In [42]:
### Evaluate colocalization results
### Load the results saved in R2 script and check out some summaries

# Libraries

In [43]:
source('MS1_Libraries.r')

# Parameters

In [44]:
path<-""
outdir<-""

In [45]:
cell_type_var = c("CD4T","CD8T","monocyte","NK","B","DC")
# c("CD4T","CD8T","monocyte","NK","B","DC")

# Data 

## Summaries

In [46]:
### EQTL coloclaization summaries

In [47]:
eqtl_summary = read.table(paste0(path, '/colocalization_results/EQTL_summary_update.csv'), row.names = NULL, sep = ",", header = TRUE)

In [48]:
eqtl_summary$X = NULL

In [49]:
nrow(eqtl_summary)

[1] 648

In [50]:
head(eqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.561000e+03,Crohn's Disease,DC1MB_TMEM176A
2,PP.H0.abf,7.520034e-04,Crohn's Disease,DC1MB_TMEM176A


In [51]:
unique(eqtl_summary$trait)

[1] "Crohn's Disease"            "Inflammatory Bowel Disease"
[3] "Multiple Sclerosis"         "Rheumatoid Arthritis"      
[5] "Asthma"                     "Type_1_Diabetes"

In [52]:
eqtl_summary = unique(eqtl_summary)

In [53]:
duplicates = eqtl_summary %>% group_by(parameter, trait, identifier) %>% count() %>% filter(n>= 2)

In [54]:
unique(duplicates$trait)

character(0)

In [55]:
# Co-EQTL colocalization summaries

In [56]:
coeqtl_summary = read.table(paste0(path, '/colocalization_results/COEQTL_summary_update.csv'), row.names = NULL, sep = ",", header = TRUE)

In [57]:
coeqtl_summary$X = NULL

In [58]:
nrow(coeqtl_summary)

[1] 44688

In [59]:
head(coeqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.560000e+03,Crohn's Disease,monocyte_TMEM176A___CAPG__TMEM176A
2,PP.H0.abf,2.350241e-02,Crohn's Disease,monocyte_TMEM176A___CAPG__TMEM176A


In [60]:
coeqtl_summary = unique(coeqtl_summary)

In [61]:
duplicates = coeqtl_summary %>% group_by(parameter, trait, identifier) %>% count() %>% filter(n>= 2)

In [62]:
unique(duplicates$trait)

character(0)

In [63]:
#unique(duplicates$parameter)

In [64]:
head(duplicates,2)

parameter,trait,identifier,n
<chr>,<chr>,<chr>,<int>


In [65]:
### Extract egene

In [66]:
coeqtl_summary$egene = str_extract(coeqtl_summary$identifier, '.*___')
coeqtl_summary$egene = str_replace(coeqtl_summary$egene, '___', '')

# Evaluate summaries

## For EQTLs

In [67]:
head(eqtl_summary,2)

,parameter,value,trait,identifier
,<chr>,<dbl>,<chr>,<chr>
1,nsnps,2.561000e+03,Crohn's Disease,DC1MB_TMEM176A
2,PP.H0.abf,7.520034e-04,Crohn's Disease,DC1MB_TMEM176A


In [68]:
## Extract amount of SNPS

In [69]:
n_snps = eqtl_summary[eqtl_summary$parameter == 'nsnps',]
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [70]:
#eqtl_summary = eqtl_summary[eqtl_summary$parameter != 'nsnps',]

In [71]:
## Extract cell-type, gene etc

In [72]:
eqtl_summary$cell_type = str_replace(eqtl_summary$identifier, '_.*', '')

In [73]:
eqtl_summary$gene = str_replace(eqtl_summary$identifier, '.*_', '')

In [74]:
unique(eqtl_summary$cell_type)

[1] "DC1MB"       "B1MB"        "NK1MB"       "monocyte1MB" "CD8T1MB"    
[6] "CD4T1MB"

In [75]:
## Add number of SNPS

In [76]:
eqtl_summary = merge(eqtl_summary, n_snps, by.x = c('trait', 'identifier'), by.y = c('trait', 'identifier'))

In [77]:
head(eqtl_summary,2)

,trait,identifier,parameter,value,cell_type,gene,overlapping_snps
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
1,Asthma,B1MB_RNASET2,nsnps,1133.0000000,B1MB,RNASET2,1133
2,Asthma,B1MB_RNASET2,PP.H0.abf,0.8915183,B1MB,RNASET2,1133


In [78]:
### Maximum probability per identifier and trait

In [79]:
max_prob = eqtl_summary[eqtl_summary$parameter != 'nsnps',] %>% group_by(trait, identifier, cell_type, gene) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier', 'cell_type'. You can
override using the `.groups` argument.


In [80]:
eqtl_summary_filtered = merge(eqtl_summary, max_prob)

In [82]:
overview_h_amounts = eqtl_summary_filtered %>% group_by(parameter, trait) %>% summarise(n = n(), mean_value = mean(value), amount_greater_0.9 = sum(value > 0.9), amount_greater_0.75 = sum(value > 0.75),amount_greater_0.5 = sum(value > 0.5))

`summarise()` has grouped output by 'parameter'. You can override using the
`.groups` argument.


In [83]:
## Add snp information

In [84]:
n_snps = n_snps %>% group_by(trait) %>% summarise(average_overlapping_snps = mean(overlapping_snps), min_overlapping_snps = min(overlapping_snps), max_overlapping_snps = max(overlapping_snps))

In [85]:
#n_snps = unique(n_snps[,c('trait', 'overlapping_snps')])

In [86]:
overview_h_amounts = merge(overview_h_amounts, n_snps, by.x = c('trait'), by.y = c('trait'))

In [87]:
#head(n_snps,3)

In [88]:
head(overview_h_amounts[order(overview_h_amounts$trait, decreasing = TRUE),],5)

,trait,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,average_overlapping_snps,min_overlapping_snps,max_overlapping_snps
,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
21,Type_1_Diabetes,PP.H2.abf,5,0.8546132,2,4,5,2487.000,1341,3161
22,Type_1_Diabetes,PP.H0.abf,7,0.6675308,0,0,6,2487.000,1341,3161
23,Type_1_Diabetes,PP.H3.abf,6,0.9997167,6,6,6,2487.000,1341,3161
16,Rheumatoid Arthritis,PP.H2.abf,2,0.8751799,0,2,2,1603.667,882,2031
17,Rheumatoid Arthritis,PP.H4.abf,6,0.7536935,0,6,6,1603.667,882,2031


In [89]:
### Inspect interesting hypothesis

In [90]:
parameter_var = 'PP.H4.abf'

In [92]:
eqtl_summary_filtered[(eqtl_summary_filtered$parameter == parameter_var) ,]

,trait,identifier,value,cell_type,gene,parameter,overlapping_snps
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
2,Asthma,B1MB_RPS26,0.5349101,B1MB,RPS26,PP.H4.abf,381
5,Asthma,CD4T1MB_RPS26,0.5349363,CD4T1MB,RPS26,PP.H4.abf,381
8,Asthma,CD8T1MB_RPS26,0.5350001,CD8T1MB,RPS26,PP.H4.abf,381
11,Asthma,DC1MB_RPS26,0.5349096,DC1MB,RPS26,PP.H4.abf,381
14,Asthma,monocyte1MB_RPS26,0.5349095,monocyte1MB,RPS26,PP.H4.abf,381
17,Asthma,NK1MB_RPS26,0.5429194,NK1MB,RPS26,PP.H4.abf,381
22,Crohn's Disease,CD4T1MB_RNASET2,0.9046377,CD4T1MB,RNASET2,PP.H4.abf,2707
25,Crohn's Disease,CD8T1MB_RNASET2,0.5453104,CD8T1MB,RNASET2,PP.H4.abf,2707
40,Inflammatory Bowel Disease,CD4T1MB_RNASET2,0.9309073,CD4T1MB,RNASET2,PP.H4.abf,2704


In [93]:
parameter_var = 'PP.H3.abf'

In [94]:
#trait = c('Rheumatoid Arthritis', 'Type_1_Diabetes', 'Crohn\'s Disease')

In [95]:
eqtl_summary_filtered[(eqtl_summary_filtered$parameter == parameter_var) ,]

,trait,identifier,value,cell_type,gene,parameter,overlapping_snps
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
34,Crohn's Disease,NK1MB_RNASET2,0.5016812,NK1MB,RNASET2,PP.H3.abf,2707
43,Inflammatory Bowel Disease,CD8T1MB_RNASET2,0.6274067,CD8T1MB,RNASET2,PP.H3.abf,2704
52,Inflammatory Bowel Disease,NK1MB_RNASET2,0.4851461,NK1MB,RNASET2,PP.H3.abf,2704
76,Rheumatoid Arthritis,CD4T1MB_RNASET2,1.0000000,CD4T1MB,RNASET2,PP.H3.abf,2031
79,Rheumatoid Arthritis,CD8T1MB_RNASET2,0.9994734,CD8T1MB,RNASET2,PP.H3.abf,2031
88,Rheumatoid Arthritis,NK1MB_RNASET2,0.7544265,NK1MB,RNASET2,PP.H3.abf,2031
92,Type_1_Diabetes,B1MB_RPS26,0.9998778,B1MB,RPS26,PP.H3.abf,1341
95,Type_1_Diabetes,CD4T1MB_RPS26,0.9997676,CD4T1MB,RPS26,PP.H3.abf,1341
98,Type_1_Diabetes,CD8T1MB_RPS26,0.9995874,CD8T1MB,RPS26,PP.H3.abf,1341


## For Co-EQTLs

### General summaries

In [96]:
head(coeqtl_summary,2)

,parameter,value,trait,identifier,egene
,<chr>,<dbl>,<chr>,<chr>,<chr>
1,nsnps,2.560000e+03,Crohn's Disease,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A
2,PP.H0.abf,2.350241e-02,Crohn's Disease,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A


In [97]:
## extract amount of SNPs info

In [98]:
n_snps = coeqtl_summary[coeqtl_summary$parameter == 'nsnps',]

In [99]:
n_snps$overlapping_snps = n_snps$value
n_snps$parameter = NULL
n_snps$value = NULL

In [100]:
## Extract relevant columsn

In [101]:
coeqtl_summary = coeqtl_summary[coeqtl_summary$parameter != 'nsnps',]

In [102]:
coeqtl_summary$cell_type = str_replace(coeqtl_summary$identifier, '_.*', '')

In [103]:
#tail(coeqtl_summary,2)

In [104]:
unique(coeqtl_summary$cell_type)

[1] "monocyte" "CD8T"     "CD4T"     "DC"       "B"        "NK"

In [105]:
coeqtl_summary$gene = str_extract(coeqtl_summary$identifier, '_.*')
coeqtl_summary$gene = str_replace(coeqtl_summary$gene, '.*___', '')

In [106]:
length(unique(coeqtl_summary$gene))

[1] 604

In [107]:
### Add number of snps

In [108]:
head(n_snps,2)

,trait,identifier,egene,overlapping_snps
,<chr>,<chr>,<chr>,<dbl>
1,Crohn's Disease,monocyte_TMEM176A___CAPG__TMEM176A,monocyte_TMEM176A,2560
7,Crohn's Disease,monocyte_TMEM176A___PTAFR__TMEM176A,monocyte_TMEM176A,2560


In [109]:
coeqtl_summary = merge(coeqtl_summary, n_snps, by.x = c('trait', 'identifier', 'egene'), by.y = c('trait', 'identifier', 'egene'))

In [110]:
### Check out maximum probabilities

In [111]:
max_value = coeqtl_summary %>% group_by(trait, identifier, cell_type, gene, overlapping_snps, egene) %>% summarise(value = max(value))

`summarise()` has grouped output by 'trait', 'identifier', 'cell_type', 'gene',
'overlapping_snps'. You can override using the `.groups` argument.


In [112]:
coeqtl_summary_filtered = merge(coeqtl_summary, max_value)

In [113]:
head(coeqtl_summary_filtered,2)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
1,Asthma,B_RPS26___EEF1A1__RPS26,B_RPS26,0.3734713,B,EEF1A1__RPS26,381,PP.H0.abf
2,Asthma,B_RPS26___RPL10__RPS26,B_RPS26,0.6338403,B,RPL10__RPS26,381,PP.H4.abf


In [114]:
overview_h_amounts = coeqtl_summary_filtered %>% group_by(parameter, trait, egene ) %>% summarise(n = n(), mean_value = mean(value), amount_greater_0.9 = sum(value > 0.9),  amount_greater_0.75 = sum(value > 0.75),amount_greater_0.5 = sum(value > 0.5),  max_overlap_snps = max(overlapping_snps), min_overlap_snps = min(overlapping_snps), mean_overlap_snps = mean(overlapping_snps))

`summarise()` has grouped output by 'parameter', 'trait'. You can override
using the `.groups` argument.


In [115]:
overview_h_amounts = overview_h_amounts[order(overview_h_amounts$trait),]

In [116]:
amount_coegenes_per_e_gene = overview_h_amounts %>% group_by(egene, trait) %>% summarise(total_n = sum(n))

`summarise()` has grouped output by 'egene'. You can override using the
`.groups` argument.


In [117]:
overview_h_amounts = merge(overview_h_amounts, amount_coegenes_per_e_gene)

In [118]:
#### Inspect the results for eGenes

In [119]:
### All RPS26 - co-egene examples

In [121]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RPS26', 'CD8T_RPS26', 'monocyte_RPS26', 'DC_RPS26', 'NK_RPS26','B_RPS26')) & (overview_h_amounts$parameter %in% c('PP.H4.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
2,Asthma,B_RPS26,PP.H4.abf,20,0.5448488,0,0,16,381,381,381.0000,35
8,Asthma,CD4T_RPS26,PP.H4.abf,353,0.6166249,0,28,323,381,368,380.7734,372
14,Asthma,CD8T_RPS26,PP.H4.abf,241,0.6293361,0,31,212,381,367,380.6307,293
24,Asthma,monocyte_RPS26,PP.H4.abf,127,0.6724475,0,40,114,381,379,380.9685,132
28,Asthma,NK_RPS26,PP.H4.abf,84,0.6149269,0,7,78,379,379,379.0000,96
105,Rheumatoid Arthritis,B_RPS26,PP.H4.abf,33,0.7385906,4,19,31,878,876,877.9394,35
111,Rheumatoid Arthritis,CD4T_RPS26,PP.H4.abf,370,0.8044940,41,325,368,882,835,881.1541,372
118,Rheumatoid Arthritis,CD8T_RPS26,PP.H4.abf,289,0.8303538,64,247,286,882,831,880.4567,293
123,Rheumatoid Arthritis,DC_RPS26,PP.H4.abf,3,0.6804751,0,0,3,855,855,855.0000,3


In [123]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RPS26', 'CD8T_RPS26', 'monocyte_RPS26', 'DC_RPS26', 'NK_RPS26','B_RPS26')) & (overview_h_amounts$parameter %in% c( 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
110,Rheumatoid Arthritis,CD4T_RPS26,PP.H3.abf,2,0.6476383,0,0,2,882,882,882.0000,372
133,Type_1_Diabetes,B_RPS26,PP.H3.abf,24,0.7354662,7,12,20,1335,1335,1335.0000,35
139,Type_1_Diabetes,CD4T_RPS26,PP.H3.abf,336,0.9025018,234,282,329,1341,1274,1339.9732,372
144,Type_1_Diabetes,CD8T_RPS26,PP.H3.abf,266,0.9176685,204,223,258,1341,1268,1339.0113,293
150,Type_1_Diabetes,DC_RPS26,PP.H3.abf,1,0.4908023,0,0,0,1297,1297,1297.0000,3
154,Type_1_Diabetes,monocyte_RPS26,PP.H3.abf,122,0.9210668,92,109,119,1341,1334,1340.9344,132
159,Type_1_Diabetes,NK_RPS26,PP.H3.abf,91,0.9621223,81,87,90,1334,1334,1334.0000,96
161,Type_1_Diabetes_SNPs_filtered_RA,B_RPS26,PP.H3.abf,17,0.7609857,5,9,16,878,878,878.0000,35
168,Type_1_Diabetes_SNPs_filtered_RA,CD4T_RPS26,PP.H3.abf,314,0.8941315,215,253,302,882,840,881.4522,372


In [124]:
### All HLA-DQA2  - co-egene examples

In [125]:
#unique(overview_h_amounts$egene )

In [126]:
overview_h_amounts[(overview_h_amounts$egene %in% c('CD4T_HLA-DQA2' , 'CD8T_HLA-DQA2', 'monocyte_HLA-DQA2', 'DC_HLA-DQA2'))  & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
3,Asthma,CD4T_HLA-DQA2,PP.H4.abf,13,0.8605121,5,11,13,387,387,387.0000,16
11,Asthma,CD8T_HLA-DQA2,PP.H4.abf,7,0.8719882,5,5,7,386,386,386.0000,7
18,Asthma,DC_HLA-DQA2,PP.H4.abf,5,0.6734167,0,1,5,358,348,351.8000,13
20,Asthma,monocyte_HLA-DQA2,PP.H4.abf,4,0.7193509,1,2,4,388,388,388.0000,17
32,Crohn's Disease,CD4T_HLA-DQA2,PP.H3.abf,16,0.8674851,8,14,16,4525,4525,4525.0000,16
39,Crohn's Disease,CD8T_HLA-DQA2,PP.H3.abf,7,0.8685034,2,6,7,4491,4383,4475.5714,7
45,Crohn's Disease,DC_HLA-DQA2,PP.H3.abf,9,0.6909695,0,3,9,3930,3758,3810.6667,13
47,Crohn's Disease,monocyte_HLA-DQA2,PP.H3.abf,17,0.9558094,14,17,17,4436,4331,4363.8235,17
57,Inflammatory Bowel Disease,CD4T_HLA-DQA2,PP.H3.abf,16,0.7763020,1,9,16,4527,4527,4527.0000,16


In [127]:
### All SMDT1  - co-egene examples

In [129]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_SMDT1', 'CD8T_SMDT1')) & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>


In [131]:
### All TMEM176A  - co-egene examples

In [132]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('monocyte_TMEM176A'))  & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>


In [133]:
### All RNASET2 - co-egene examples

In [134]:
overview_h_amounts[(overview_h_amounts$egene %in%  c('CD4T_RNASET2', 'monocyte_RNASET2')) & (overview_h_amounts$parameter %in% c('PP.H4.abf', 'PP.H3.abf')),]

,trait,egene,parameter,n,mean_value,amount_greater_0.9,amount_greater_0.75,amount_greater_0.5,max_overlap_snps,min_overlap_snps,mean_overlap_snps,total_n
,<chr>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
33,Crohn's Disease,CD4T_RNASET2,PP.H3.abf,2,0.5193770,0,0,1,2707,2707,2707,4
34,Crohn's Disease,CD4T_RNASET2,PP.H4.abf,2,0.5193536,0,0,2,2707,2707,2707,4
48,Crohn's Disease,monocyte_RNASET2,PP.H3.abf,1,0.8485668,0,1,1,4390,4390,4390,1
58,Inflammatory Bowel Disease,CD4T_RNASET2,PP.H3.abf,4,0.5968840,0,0,4,2704,2704,2704,4
72,Inflammatory Bowel Disease,monocyte_RNASET2,PP.H3.abf,1,0.8485287,0,1,1,4391,4391,4391,1
109,Rheumatoid Arthritis,CD4T_RNASET2,PP.H3.abf,4,0.9369664,3,4,4,2031,2031,2031,4


In [135]:
#overview_h_amounts[order(overview_h_amounts$trait),]

### Get certain co-eqtl examples (RA - RPS26 H4 coegenes)

In [136]:
## 41 co-eGenes for Rheumatoid Arthritis with strong colocalization signal

In [137]:
coeqtl_summary_filtered$gene = str_replace(coeqtl_summary_filtered$gene, '_', '')

In [138]:
nrow(coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ),])

[1] 152

In [139]:
coloc_examples_rheomatoid_arthritis = coeqtl_summary_filtered[(coeqtl_summary_filtered$trait == 'Rheumatoid Arthritis' ) & (coeqtl_summary_filtered$value > 0.9 )& (coeqtl_summary_filtered$parameter == 'PP.H4.abf' ) & (coeqtl_summary_filtered$egene == 'CD4T_RPS26' ),]

In [140]:
head(coloc_examples_rheomatoid_arthritis[order(coloc_examples_rheomatoid_arthritis$value, decreasing = TRUE),],6)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>
4428,Rheumatoid Arthritis,CD4T_RPS26___FOXP3__RPS26,CD4T_RPS26,0.9669022,CD4T,FOXP3_RPS26,870,PP.H4.abf
4451,Rheumatoid Arthritis,CD4T_RPS26___HLA-DRA__RPS26,CD4T_RPS26,0.9651487,CD4T,HLA-DRA_RPS26,882,PP.H4.abf
4382,Rheumatoid Arthritis,CD4T_RPS26___COX7C__RPS26,CD4T_RPS26,0.9617738,CD4T,COX7C_RPS26,882,PP.H4.abf
4312,Rheumatoid Arthritis,CD4T_RPS26___ABHD14B__RPS26,CD4T_RPS26,0.9584893,CD4T,ABHD14B_RPS26,882,PP.H4.abf
4321,Rheumatoid Arthritis,CD4T_RPS26___AKAP13__RPS26,CD4T_RPS26,0.9574507,CD4T,AKAP13_RPS26,882,PP.H4.abf
4486,Rheumatoid Arthritis,CD4T_RPS26___MIR4435-1HG__RPS26,CD4T_RPS26,0.9515798,CD4T,MIR4435-1HG_RPS26,882,PP.H4.abf


In [141]:
coloc_examples_rheomatoid_arthritis %>% group_by(egene) %>% count()

egene,n
<chr>,<int>
CD4T_RPS26,41


In [142]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 41

In [143]:
### Mapp positions for coegenes

In [144]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$gene, 'RPS26', '')

In [145]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '__', '')

In [146]:
coloc_examples_rheomatoid_arthritis$cogene = str_replace(coloc_examples_rheomatoid_arthritis$cogene, '_', '')

In [147]:
unique(coloc_examples_rheomatoid_arthritis$cogene)

[1] "ABHD14B"     "ADAM19"      "AIF1"        "AKAP13"      "ATP2B1"     
 [6] "ATP2B4"      "C12orf75"    "CD58"        "CHCHD2"      "COX7C"      
[11] "CST7"        "CYBA"        "EML4"        "ENTPD1"      "FOXP3"      
[16] "FTH1"        "GALM"        "GK"          "H3F3A"       "HLA-DPB1"   
[21] "HLA-DRA"     "IGBP1"       "ISG20"       "LINC00493"   "LSM5"       
[26] "MIAT"        "MIR4435-1HG" "MT2A"        "PRMT2"       "RGS1"       
[31] "RNF19A"      "SBDS"        "SERF2"       "TCF7"        "TIGIT"      
[36] "TMA7"        "TMSB10"      "TNFRSF1B"    "UBB"         "YBX1"       
[41] "YWHAB"

In [148]:
### Retrieve gene positions an map to the data

In [149]:
mart = useEnsembl(biomart = "genes", dataset = "hsapiens_gene_ensembl")

In [150]:
geneSet =  unique(coloc_examples_rheomatoid_arthritis$cogene)

In [151]:
resultTable = biomaRt::getBM(attributes = c("start_position","end_position","description", 'hgnc_symbol', 'chromosome_name'),       
                  filters    = "hgnc_symbol",       
                  values     = geneSet,         
                  mart       = mart)     

In [152]:
nrow(resultTable)

[1] 57

In [153]:
length(unique(resultTable$hgnc_symbol))

[1] 38

In [154]:
### Filter out duplicate mappings

In [155]:
filter = resultTable %>% group_by(hgnc_symbol) %>% count() %>% filter(n >= 2)

In [156]:
filter = filter$hgnc_symbol

In [157]:
resultTable = resultTable[!resultTable$hgnc_symbol %in% filter,]

In [158]:
unique(resultTable$chromosome_name)

[1] "3"  "5"  "15" "12" "1"  "7"  "20" "16" "2"  "10" "X"  "11" "22" "21" "8" 
[16] "17"

In [159]:
head(resultTable,2)

,start_position,end_position,description,hgnc_symbol,chromosome_name
,<int>,<int>,<chr>,<chr>,<chr>
1,51968510,51983409,abhydrolase domain containing 14B [Source:HGNC Symbol;Acc:HGNC:28235],ABHD14B,3
2,157395534,157575775,ADAM metallopeptidase domain 19 [Source:HGNC Symbol;Acc:HGNC:197],ADAM19,5


In [160]:
head(coloc_examples_rheomatoid_arthritis,2)

,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,cogene
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>
4312,Rheumatoid Arthritis,CD4T_RPS26___ABHD14B__RPS26,CD4T_RPS26,0.9584893,CD4T,ABHD14B_RPS26,882,PP.H4.abf,ABHD14B
4316,Rheumatoid Arthritis,CD4T_RPS26___ADAM19__RPS26,CD4T_RPS26,0.9157420,CD4T,ADAM19_RPS26,882,PP.H4.abf,ADAM19


In [161]:
coloc_examples_rheomatoid_arthritis = merge(coloc_examples_rheomatoid_arthritis, resultTable, by.x = 'cogene', by.y = 'hgnc_symbol', all.x = TRUE)

In [162]:
head(coloc_examples_rheomatoid_arthritis[is.na(coloc_examples_rheomatoid_arthritis$chromosome_name),])

,cogene,trait,identifier,egene,value,cell_type,gene,overlapping_snps,parameter,start_position,end_position,description,chromosome_name
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<int>,<chr>,<chr>
3,AIF1,Rheumatoid Arthritis,CD4T_RPS26___AIF1__RPS26,CD4T_RPS26,0.9269214,CD4T,AIF1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
19,H3F3A,Rheumatoid Arthritis,CD4T_RPS26___H3F3A__RPS26,CD4T_RPS26,0.9313711,CD4T,H3F3A_RPS26,882,PP.H4.abf,NA,NA,NA,NA
20,HLA-DPB1,Rheumatoid Arthritis,CD4T_RPS26___HLA-DPB1__RPS26,CD4T_RPS26,0.9147419,CD4T,HLA-DPB1_RPS26,882,PP.H4.abf,NA,NA,NA,NA
21,HLA-DRA,Rheumatoid Arthritis,CD4T_RPS26___HLA-DRA__RPS26,CD4T_RPS26,0.9651487,CD4T,HLA-DRA_RPS26,882,PP.H4.abf,NA,NA,NA,NA
24,LINC00493,Rheumatoid Arthritis,CD4T_RPS26___LINC00493__RPS26,CD4T_RPS26,0.9276682,CD4T,LINC00493_RPS26,882,PP.H4.abf,NA,NA,NA,NA
27,MIR4435-1HG,Rheumatoid Arthritis,CD4T_RPS26___MIR4435-1HG__RPS26,CD4T_RPS26,0.9515798,CD4T,MIR4435-1HG_RPS26,882,PP.H4.abf,NA,NA,NA,NA


In [163]:
#head(coloc_examples_rheomatoid_arthritis,4)

In [164]:
nrow(coloc_examples_rheomatoid_arthritis)

[1] 41

In [165]:
write.table(coloc_examples_rheomatoid_arthritis, file = paste0(path, "/colocalization_results/", "CD4T_RPS26_Rheomatoid_Arthritis_Colocalization_CoeGenes.csv"), append =FALSE, sep = ",", row.names = FALSE, col.names =TRUE)